In [2]:
import requests
import pandas as pd
import numpy as np
import json

In [3]:
def make_request(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        return json.loads(response.text)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None
    
#main table of responses
response = make_request("https://gbfs.capitalbikeshare.com/gbfs/2.3/gbfs.json")['data']['en']

In [11]:
#station information
stations = make_request("https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/station_information.json")['data']['stations']
#station region
station_regions = make_request('https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/system_regions.json')['data']['regions']
#json to dataframe
df = pd.DataFrame(stations)
df_region = pd.DataFrame(station_regions)
#merge dataframes
df_region = df_region.rename(columns = {'name':'region_name'})

df = pd.merge(df, df_region, left_on='region_id', right_on='region_id', how='left')

#convert lat and lon to float
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

#write data to csv
df.to_csv('stations.csv', index=False)
df.sort_values(by=['lat', 'lon'], inplace=True)
df

,name,region_id,capacity,lon,short_name,station_id,rental_uris,lat,region_name
706,Charles Arrington Dr & Manchester Lakes Dr,NaN,12,-77.150359,32290,1917259365558921792,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",38.766844,NaN
118,Franconia-Springfield Metro North,NaN,19,-77.168902,32289,1917259666206632502,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",38.766946,NaN
406,Metro Park Dr & Walker Ln,NaN,12,-77.156504,32296,1957669092295526720,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",38.768430,NaN
166,Seatrend Way & Topsails Ln,NaN,12,-77.160014,32288,1917259953969440770,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",38.775639,NaN
419,Fleet St & Waterfront St,133,15,-77.016059,32406,d4e8307d-5fb2-4ece-a66a-1522b1ebf79c,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",38.782633,Prince George's County
...,...,...,...,...,...,...,...,...,...
129,Shady Grove Metro West,43,19,-77.166093,32045,0825e56c-1f3f-11e7-bf6b-3863bb334450,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",39.119765,"Montgomery County, MD (North)"
603,Shady Grove Metro East,43,19,-77.164969,32067,08265563-1f3f-11e7-bf6b-3863bb334450,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",39.121327,"Montgomery County, MD (North)"
684,Needwood Rd & Eagles Head Ct,43,15,-77.157410,32018,0825962a-1f3f-11e7-bf6b-3863bb334450,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",39.123513,"Montgomery County, MD (North)"
654,Crabbs Branch Avenue & Gramercy Blvd,43,11,-77.164467,32095,2069782102977517916,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",39.124921,"Montgomery County, MD (North)"


In [9]:
#free bike status
free_bike_status = make_request("https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/free_bike_status.json")['data']['bikes']
#vehicle type
vehicle_types = make_request("https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/vehicle_types.json")['data']['vehicle_types']
#json to dataframe
df_free_bikes = pd.DataFrame(free_bike_status)
df_vehicle_types = pd.DataFrame(vehicle_types)
#join dataframes on vehicle_type_id
df_free_bikes = df_free_bikes.merge(df_vehicle_types, on='vehicle_type_id', how='left')
#clean up dataframe
#df_free_bikes = df_free_bikes[['bike_id', 'lat', 'lon']]
#convert lat and lon to float
df_free_bikes['lat'] = df_free_bikes['lat'].astype(float)
df_free_bikes['lon'] = df_free_bikes['lon'].astype(float)
#write data to csv
df_free_bikes.to_csv('free_bikes.csv', index=False)
df_free_bikes

,is_disabled,lon,vehicle_type_id,current_range_meters,bike_id,lat,is_reserved,rental_uris,propulsion_type,form_factor,max_range_meters
0,0,-77.052967,2,29129.1264,aa841f9bb44ba9164c8def6c85a5361c,38.878894,0,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",electric_assist,bicycle,55844.24
1,0,-76.896785,2,28485.3888,8d6cfb9cb35c572f801b660bf0c3d951,39.006923,0,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",electric_assist,bicycle,55844.24
2,0,-77.084035,2,4023.3600,1fbc4546ba504786f9a43d5d61201cdb,38.863398,0,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",electric_assist,bicycle,55844.24
3,0,-76.961806,2,14162.2272,d500484eab11a72b5a58881d4cd46244,38.876697,0,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",electric_assist,bicycle,55844.24
4,0,-76.937693,2,5471.7696,0239aabcaaa1899bdc51234f087a6148,38.894745,0,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",electric_assist,bicycle,55844.24
...,...,...,...,...,...,...,...,...,...,...,...
907,0,-76.921966,2,33796.2240,84a29e198362b3b1f7b3173dde347b13,38.900378,0,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",electric_assist,bicycle,55844.24
908,0,-77.005585,2,54234.8928,9ee0cfa25307c9caf89ef9891490f9ec,38.822624,0,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",electric_assist,bicycle,55844.24
909,0,-77.018332,2,1287.4752,7ea50082eaa8975de2bac29fcc2b0d8c,38.930425,0,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",electric_assist,bicycle,55844.24
910,0,-77.076123,2,23013.6192,651c8f34e9e3a2f2451c405c8fa635bf,38.890748,0,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",electric_assist,bicycle,55844.24
